<div class="alert alert-success">
    -----
</div>

# Abstract

# Introduction

# `Analysis`

## Imports

## Behavioral Analysis

 >• Do participants perform better than chance?

> • Do they improve over trials within each round?

> • Do they improve over rounds?

> • How does the number of observed features affect performance?

> • How does time pressure affect performance?

## Parameter Fitting

`Look at the resulting parameters and try to answer the following questions`

### Recorla-Wagner

> • Does time pressure lead to slower learning?

> • Does time pressure lead to more noisy decisions?

### Kalman Filter

> • Does time pressure lead to slower learning?

> • Does time pressure lead to more noisy decisions?

### Error Model

> • Does time pressure lead to slower learning?

> • Does time pressure lead to more noisy decisions?

### <.....> Model

> • Does time pressure lead to slower learning?

> • Does time pressure lead to more noisy decisions?

## Model Comparison

> •  Which model explains the human data best?

> • Does time pressure lead to simpler learning strategies?

# Methods

# Results

# Discussion

# References